# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, behavior

Connecting cbroz@dss-db.datajoint.io:3306


Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.conn()

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [2]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

Root: ['/Users/cb/Documents/data/Bonn/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C']
Full: /Users/cb/Documents/data/Bonn/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat


### Initial check of tables

In [3]:
from adamacs.pipeline import session, event, trial

session.Session.delete()
print('Sessions:', len(session.Session()))
print('Trials  :', len(trial.Trial()))
print('Events  :', len(event.Event()))

Deleting 0 rows from `bonn_session`.`session`
Nothing to delete.
Sessions: 0
Trials  : 0
Events  : 0


## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [4]:
bpod_path = "scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
bpod_object = Bpodfile(bpod_path)
bpod_object.ingest()

/Users/cb/miniforge3/envs/bon/lib/python3.8/site-packages/pywavesurfer/ws.py:46: RuntimeWarning: You are reading a WaveSurfer file version this module was not tested with: file version 1.0.5, latest version tested: 0.97
  warnings.warn('You are reading a WaveSurfer file version this module was not tested with: '


BPod items to be inserted:
	Subject : WEZ-8701
	Time    : 2022-03-18 16:55:28
	N Trials: 6
	N Events: 18


Check that insertion worked:

In [5]:
trial.TrialEvent & 'trial_id=1'

session_id,trial_id trial number (1-based indexing),event_type,event_start_time (second) relative to recording start
1,1,at_port,39.5742
1,1,at_target,33.7099
1,1,cue,33.6998
1,1,drinking,41.0177
1,1,in_port_2,40.8487


We can also interact with bpod objects. For example:

In [6]:
bpod_object.trial(1).attributes

{'error': False, 'timeout': False}

In [7]:
bpod_object.trial(1).events

{'cue': 1.2645,
 'at_target': 1.2745,
 'at_port': 7.1388,
 'in_port_2': 8.4133,
 'drinking': 8.5823}

# Add Harp recording

In [2]:
from adamacs.pipeline import behavior, event
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

session_id,harp_device_id


In [3]:
behavior.HarpRecording.populate()

In [4]:
behavior.HarpRecording.Channel()

session_id,harp_device_id,channel_name,data 1d array of acquired data for this channel,time 1d array of timestamps for this channel
1,1,DI0,=BLOB=,=BLOB=
1,1,DI1,=BLOB=,=BLOB=
1,1,Register 34,=BLOB=,=BLOB=
1,1,Register 35,=BLOB=,=BLOB=
